In [1]:
import pandas as pd
import numpy as np

# Categorical 데이터

In [2]:
values = pd.Series([0,1,0,0]*2)
dim = pd.Series(['apple','orange'])

# take 메서드를 사용하면 Series 내에 저장된 원래 문자열을 구할 수 있다.(apple이 0, orange가 1을 갖기로 한다.)
dim.take(values)   # values의 값들을 새로운 인덱스 로우로써 사용

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

pandas의 Categorical

In [13]:
fruits = ['apple','orange','apple','apple']*2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits, 'basket_id':np.arange(N) , 'count':np.random.randint(3,15,size=N),
                  'weight':np.random.uniform(0,4,size=N)}, columns=['basket_id','fruit','count','weight'])
df

,basket_id,fruit,count,weight
0,0,apple,5,0.089583
1,1,orange,13,2.543610
2,2,apple,11,1.994375
3,3,apple,5,3.031547
4,4,apple,9,0.158186
5,5,orange,13,1.595457
6,6,apple,9,3.801846
7,7,apple,3,0.496286


In [7]:
# df['fruits']는 파이썬 문자열 객체의 배열임. 쉽게 범주형 데이터로 변경할 수 있다.

fruit_cat = df['fruit'].astype('category')       # astype 주목, 괄호()안의 객체로 변경해줌. 
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [12]:
c = fruit_cat.values
type(c)
# fruit_cat의 값은 NumPy 배열이 아니라 pandas.Categorical의 인스턴스다.

pandas.core.arrays.categorical.Categorical

In [17]:
# 'Categorical 객체'는 categories와 codes 속성을 가진다.
c.categories

Index(['apple', 'orange'], dtype='object')

In [18]:
c.codes            # c에는 index 속성이 없다. == c.index-->에러,    출력: apple,orange,apple,apple,..순

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [19]:
# 변경 완료된 값을 대입함으로써 DataFrame의 컬럼을 범주형으로 변경할 수 있다.

df['fruit'] = df['fruit'].astype('category')
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [20]:
# 직접 pd.Categorical 형을 생산하는 것도 가능하다.

my_categories = pd.Categorical(['foo','bar','baz','foo','bar'])
my_categories     # 출력값을 보면 알겠지만 명시적으로 지정하지 않는 한 카테고리의 특정 순서를 보장하지 않는다.

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [21]:
# 기존에 정의된 범주와 코드로부터 생성하는 것도 가능

categories = ['foo','bar','baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes,categories)  # from_codes 함수 기억

my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [22]:
# categories 배열은 입력 데이터의 순서에 따라 다른 순서로 나타날 수 있다.
# from_codes를 사용하거나 다른 범주형 데이터 생성자를 이용하는 경우 순서를 지정할 수 있다.

ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)     # ordered 옵션
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [23]:
# 순서가 없는 범주형 인스턴스도 as_ordered 메서드를 이용해 순서를 가지도록 만들 수 있다.

my_cats_2.as_ordered()   # as_ordered 함수

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

Categorical 연산

In [24]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [25]:
# 임의의 숫자 데이터를 pandas.qcut or cut 함수로 구분한다면 pandas.Categorical 객체를 반환한다.

bins = pd.qcut(draws, 4)
bins    # bins는 categorical 인스턴스

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [26]:
bins = pd.qcut(draws, 4,labels=['Q1','Q2','Q3','Q4'])   # labels인자로 categories의 이름들을 바꿀 수 있음
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [27]:
bins.codes[:6]

array([1, 2, 1, 1, 3, 3], dtype=int8)

In [28]:
bins = pd.Series(bins, name='quartile')       # 생성된 Series는 카테고리를 가지고 있음
results = pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index()
# reset_index로 원래 index이던 quartile을 컬럼으로 이동시키고 기본 index를 추가함
# 애초에 bins를 Series로 바꾸지 않아도 DataFrame이 출력되지만 컬럼이름이 그냥 index

results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [29]:
results['quartile']
# quartile컬럼은 bins의 순서를 포함한 원래의 범주 정보를 유지하고 있음

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

categorical을 이용한 성능 개선

In [30]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux']*(N//4))

categories = labels.astype('category')

# categories객체가 일반 Series에 비해 훨씬 적은 메모리를 사용한다.
print(labels.memory_usage())
print(categories.memory_usage())

80000128
10000320


In [31]:
# 범주형으로 변환하는 과정 역시 그냥 이루지지는 않지만 이는 단 한번만 변환하면 되는 일회성 비용이다.
%time _ =labels.astype('category')

Wall time: 691 ms


Categorical 메서드

In [32]:
s = pd.Series(['a','b','c','d']*2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [33]:
# cat_s는 범주형 데이터를 담고 있는 Series

# 특별한 속성인 cat을 통해 categorical 메서드에 접근할 수 있다.

cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [34]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [35]:
# cat의 set_categories 메서드를 이용해 변경하는 것도 가능하다

actual_categories = ['a','b','c','d','e']
cat_s2 = cat_s.cat.set_categories(actual_categories)  # 데이터는 변함 없지만 새 카테고리가 추가되었다.(아래 두 셀 참고)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [36]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [37]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [38]:
# 큰 DataFrame이나 Series를 한 번 걸러내고 나면 실제로 데이터에는 존재하지 않는 카테고리가 남아있을 수 있다.

cat_s3 = cat_s[cat_s.isin(['a','b'])]   #cat_s[불리언 배열(데이터를 반환하는 기준)] (10,1,4,5번 로우만 반환)
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [39]:
cat_s3.cat.remove_unused_categories()   # remove_unused_categories 메서드를 이용해 관측되지 않는 카테고리를 제거할 수 있다.

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

모델링을 위한 더미값 생성하기

In [40]:
cat_s = pd.Series(['a','b','c','d']*2,dtype='category')  # 이런 카테고리 객체에도 get_dummies 함수 사용가능
pd.get_dummies(cat_s)  # 7장 참고, 일종의 불리언 배열로 생각해도 좋을 듯

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 고급 GroupBy 사용

그룹 변환과 GroupBy 객체 풀어내기

In [72]:
df = pd.DataFrame({'key':['a','b','c']*4, 'value':np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [73]:
# key에 따른 그룹의 평균
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [43]:
# df['value']와 같은 Series를 원한게 아니라 'key'에 따른 그룹의 평균값으로 값을 변경하기 원한다면 다음과 같이
g.transform(lambda x:x.mean())   # 이미 g는 key별로 묶인 상태

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [44]:
# 내장 요약함수에 대해서는 agg 메서드에서처럼 '문자열' 그룹 연산 이름을 넘긴다.
# mean, sum등 내장 요약함수는 일반적인 apply 함수보다 transform 에서 더 빠르게 동작한다.
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [45]:
# apply와 마찬가지로 transform은 Series를 반환하는 함수만 사용할 수 있는데 결과는 입력값과 똑같은 크기여야한다.

g.transform(lambda x: x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [46]:
# 각 그룹에 대해 내림차순으로 순위를 계산

g.transform(lambda x: x.rank(ascending=False))   # transform 자리에 apply를 넣어도 같은 결과
# 같은 key내에서의 순위를 표시한 듯(?)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

시계열 그룹 리샘플링

In [47]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time':times, 'value':np.arange(N)})

df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [48]:
df.set_index('time').resample('5min').count()
# time으로 색인한 뒤-->다운샘플링  (리샘플링은 index에서 일어남)

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [49]:
df2 = pd.DataFrame({'time':times.repeat(3), 'key':np.tile(['a','b','c'],N), 'value':np.arange(N*3.)})
                    # 같은 로우가 연속 3번 반복, np.tile도 마찬가지
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [50]:
# key의 각 값에 대해 리샘플 하기 위해 pandas.TimeGrouper 객체를 이용....해야하는데 안되네. 나중에 다시 봅시다.(p500)

#time_key = pd.TimeGrouper('5min')
#resampled = df2.set_index('time').groupby(['key',time_key]).sum()
#resampled

# 메서드 연결 기법

pipe 메서드